In [1]:
# pyDEseq2 requires:
# - numpy==1.23.0
# - anndata==0.8.0
# - pandas==1.4.3
# - scikit-learn==1.1.1
# - scipy==1.8.1
# - statsmodels==0.13.2

import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from subprocess import Popen, PIPE
from math import log
import pyensembl as ensembl
import io
ensembldb = ensembl.EnsemblRelease(release = 86)

genes = ensembldb.genes()
coding_genes = [gene for gene in genes if gene.biotype == 'protein_coding']



In [20]:
# Setup Metadata
samples = {'P1_T':'../data/SN_MO/P1_T_outs/P1_T_outs_atac_fragments.tsv.gz',
           'P1_N':'../data/SN_MO/P1_N_outs/P1_N_outs_atac_fragments.tsv.gz',
           'P2_T':'../data/SN_MO/P2_T_outs/P2_T_outs_atac_fragments.tsv.gz',
           'P2_N':'../data/SN_MO/P2_N_outs/P2_N_outs_atac_fragments.tsv.gz',
           'P3_T':'../data/SN_MO/P3_T_outs/P3_T_outs_atac_fragments.tsv.gz',
           'P4_T':'../data/SN_MO/P4_T_outs/P4_T_outs_atac_fragments.tsv.gz'}

barcodes = {}

for sample in samples.keys():
    with open('../results/%s/%s_cort_barcodes_formatted.csv' % (sample, sample), 'r') as fin:
        barcodes[sample] = [line.strip() for line in fin]


In [33]:
peaks = pd.read_csv('../results/ATAC/macs/cort_peaks.narrowPeak', sep='\t', header=None)
peaks = peaks[[0,1,2,3]]
peaks.columns = ['Chr', 'Start', 'End', 'PeakID']
peaks.index = peaks['PeakID']
peak_ids = peaks['PeakID']
queries = peak_ids.map(lambda x: '{}:{}-{}'.format(peaks.loc[x]['Chr'], peaks.loc[x]['Start'], peaks.loc[x]['End']))
queries = queries.rename('Query')
peak_key = pd.concat([peak_ids,queries], axis=1)
peak_key

,PeakID,Query
PeakID,,
cort_peak_1,cort_peak_1,GL000008.2:2217-2483
cort_peak_2,cort_peak_2,GL000008.2:2531-3604
cort_peak_3,cort_peak_3,GL000008.2:3731-3881
cort_peak_4,cort_peak_4,GL000008.2:83282-83452
cort_peak_5,cort_peak_5,GL000008.2:83571-83972
...,...,...
cort_peak_181882,cort_peak_181882,chrY:56843616-56843854
cort_peak_181883,cort_peak_181883,chrY:56849846-56849939
cort_peak_181884,cort_peak_181884,chrY:56850358-56850506


In [17]:
### Assemble Counts DF using row-wise apply function, adding columns. 
# Function goes row by row and uses the query built before and the tabix command to query the fragments file.
# Fragments are then filtered based on the corticotroph barcodes


def f(x):
    result = []
    for sample in samples.keys():
        process = Popen(['tabix', samples[sample], x['Query']], stdout=PIPE, universal_newlines=True)
        process = process.communicate()
        if process[0] == '':
            result.append(0)
        elif process[0] != '':
            frags_df = pd.read_csv(io.StringIO(process[0]), sep = '\t',header=None).drop([4], axis=1)
            frags_df = frags_df.loc[(frags_df[3].isin(barcodes[sample]))]
            result.append(len(frags_df))
    return result

In [31]:
counts = pd.DataFrame(peak_key['Query'])
          
counts[list(samples.keys())] = counts.apply(f, axis=1, result_type='expand')

counts

,Query,P1_T,P1_N,P2_T,P2_N,P3_T,P4_T
PeakID,,,,,,,
cort_peak_1,GL000008.2:2217-2483,0,0,0,0,0,0
cort_peak_2,GL000008.2:2531-3604,0,0,0,0,0,0
cort_peak_3,GL000008.2:3731-3881,0,0,0,0,0,0
cort_peak_4,GL000008.2:83282-83452,0,0,0,0,0,0
cort_peak_5,GL000008.2:83571-83972,0,0,0,0,0,0
...,...,...,...,...,...,...,...
cort_peak_181882,chrY:56843616-56843854,1,5,2,0,75,6
cort_peak_181883,chrY:56849846-56849939,2,0,1,0,9,3
cort_peak_181884,chrY:56850358-56850506,1,3,0,0,22,11


In [32]:
counts.to_csv('../results/ATAC/differential/peak_counts.csv')

In [6]:
counts = pd.read_csv('../results/ATAC/differential/peak_counts.csv', index_col=0)
counts = counts.drop('Query', axis=1)
counts

,P1_T,P1_N,P2_T,P2_N,P3_T,P4_T
PeakID,,,,,,
cort_peak_1,0,0,0,0,0,0
cort_peak_2,0,0,0,0,0,0
cort_peak_3,0,0,0,0,0,0
cort_peak_4,0,0,0,0,0,0
cort_peak_5,0,0,0,0,0,0
...,...,...,...,...,...,...
cort_peak_181882,1,5,2,0,75,6
cort_peak_181883,2,0,1,0,9,3
cort_peak_181884,1,3,0,0,22,11


In [26]:
counts = counts.transpose()
# Filter out features with low fragment counts
genes_to_keep = counts.columns[counts.sum(axis=0) >= 10]
counts = counts[genes_to_keep]
counts

PeakID,cort_peak_11,cort_peak_14,cort_peak_15,cort_peak_16,cort_peak_17,cort_peak_19,cort_peak_21,cort_peak_22,cort_peak_23,cort_peak_25,...,cort_peak_181877,cort_peak_181878,cort_peak_181879,cort_peak_181880,cort_peak_181881,cort_peak_181882,cort_peak_181883,cort_peak_181884,cort_peak_181885,cort_peak_181886
P1_T,0,0,0,0,0,2,0,0,1,0,...,1,2,4,0,1,1,2,1,1,1
P1_N,2,2,6,5,0,19,1,1,8,1,...,2,2,4,3,0,5,0,3,3,5
P2_T,21,1,2,2,3,29,1,1,11,0,...,2,3,3,5,3,2,1,0,1,4
P2_N,0,0,0,0,0,1,1,0,0,1,...,1,1,0,0,1,0,0,0,2,3
P3_T,4,0,8,4,8,7,2,0,3,26,...,16,21,26,43,32,75,9,22,38,700
P4_T,100,24,49,15,15,47,16,16,62,17,...,7,8,10,2,9,6,3,11,17,11


In [44]:
gene_queries = {}
for gene in coding_genes:
    if gene.start < gene.end:
        gene_queries[gene.gene_name] = 'chr%s:%d-%d' % (gene.contig, gene.start-5000, gene.end+5000)
    elif gene.start > gene.end:
        gene_queries[gene.gene_name] = 'chr%s:%d-%d' % (gene.contig, gene.start+5000, gene.end-5000)    
gene_counts = pd.DataFrame(list(gene_queries.values()), index=gene_queries.keys(), columns=['Query'])
gene_counts[list(samples.keys())] = gene_counts.apply(f, axis=1, result_type='expand')
gene_counts

,Query,P1_T,P1_N,P2_T,P2_N,P3_T,P4_T
TSPAN6,chrX:100622109-100644991,21,43,97,16,951,573
TNMD,chrX:100579802-100604885,12,18,30,8,231,99
DPM1,chr20:50929867-50963555,51,153,334,37,4398,1073
SCYL3,chr1:169844631-169899267,60,131,414,35,4199,1053
C1orf112,chr1:169657007-169859080,105,152,298,92,2929,756
...,...,...,...,...,...,...,...
RP13-210D15.9,chrX:135304480-135314659,2,6,11,3,117,30
RP11-244E17.1,chr14:23094065-23109992,71,231,425,41,5005,1447
CTB-60B18.23,chr19:49050793-49070076,29,60,114,16,897,236
RP3-454G6.2,chr1:171595621-171643799,31,43,116,21,644,341


In [45]:
gene_counts.to_csv('../results/ATAC/differential/gene_counts.csv')

In [46]:
gene_counts = gene_counts.drop('Query', axis=1)
gene_counts

,P1_T,P1_N,P2_T,P2_N,P3_T,P4_T
TSPAN6,21,43,97,16,951,573
TNMD,12,18,30,8,231,99
DPM1,51,153,334,37,4398,1073
SCYL3,60,131,414,35,4199,1053
C1orf112,105,152,298,92,2929,756
...,...,...,...,...,...,...
RP13-210D15.9,2,6,11,3,117,30
RP11-244E17.1,71,231,425,41,5005,1447
CTB-60B18.23,29,60,114,16,897,236
RP3-454G6.2,31,43,116,21,644,341


In [47]:
gene_counts = gene_counts.transpose()
# Filter out features with low fragment counts
genes_to_keep = gene_counts.columns[gene_counts.sum(axis=0) >= 10]
gene_counts = gene_counts[genes_to_keep]
gene_counts

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,RP5-994D16.12,RP11-402P6.15,AC074141.1,AL022397.1,AL022067.1,RP13-210D15.9,RP11-244E17.1,CTB-60B18.23,RP3-454G6.2,RP5-937E21.8
P1_T,21,12,51,60,105,45,39,22,129,78,...,62,15,6,122,60,2,71,29,31,4
P1_N,43,18,153,131,152,69,66,54,281,290,...,196,20,7,298,139,6,231,60,43,5
P2_T,97,30,334,414,298,88,86,120,544,530,...,329,22,22,890,269,11,425,114,116,4
P2_N,16,8,37,35,92,17,31,25,96,62,...,40,7,6,109,47,3,41,16,21,0
P3_T,951,231,4398,4199,2929,880,712,1291,7192,8249,...,4791,352,110,7712,1814,117,5005,897,644,42
P4_T,573,99,1073,1053,756,208,222,400,1524,2387,...,795,117,74,2127,751,30,1447,236,341,25


In [48]:
clinical_df = pd.DataFrame(['Tumor', 'Normal', 'Tumor', 'Normal', 'Tumor', 'Tumor'], ['P1_T', 'P1_N', 'P2_T', 'P2_N', 'P3_T', 'P4_T'], ["Annotation"])
clinical_df


,Annotation
P1_T,Tumor
P1_N,Normal
P2_T,Tumor
P2_N,Normal
P3_T,Tumor
P4_T,Tumor


In [50]:
## Peaks
dds = DeseqDataSet(
    counts=counts,
    clinical=clinical_df,
    design_factors="Annotation",
    refit_cooks=True,
    n_cpus=1,
)
dds


AnnData object with n_obs × n_vars = 6 × 175624
    obs: 'Annotation'
    obsm: 'design_matrix'

In [51]:
dds.deseq2()

Fitting size factors...
... done in 0.04 seconds.

Fitting dispersions...


KeyboardInterrupt: 

In [31]:
stat_res = DeseqStats(dds, n_cpus=2)
stat_res.summary()
peak_results = stat_res.results_df
peak_results = peak_results.sort_values('pvalue', ascending=True)
peak_results.to_csv('../results/ATAC/differential/DAC_peaks.csv')
peak_results

Running Wald tests...
... done in 6.18 seconds.

Log2 fold change & Wald test p-value: Annotation Tumor vs Normal


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
PeakID,,,,,,
cort_peak_11,9.184301,2.523969,2.565003,0.984002,0.325114,0.999998
cort_peak_14,2.024936,0.073297,2.349866,0.031192,0.975116,0.999998
cort_peak_15,4.751814,-0.308316,1.574977,-0.195759,0.844799,0.999998
cort_peak_16,2.598618,-1.382005,1.773673,-0.779177,0.435876,0.999998
cort_peak_17,1.328844,1.880299,3.019965,0.622623,0.533532,0.999998
...,...,...,...,...,...,...
cort_peak_181882,3.560776,-0.957155,1.420471,-0.673829,0.500420,0.999998
cort_peak_181883,2.104789,1.550597,2.822595,0.549352,0.582764,0.999998
cort_peak_181884,2.505286,-0.741095,1.814857,-0.408349,0.683017,0.999998


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
PeakID,,,,,,
cort_peak_42326,28.625864,-2.419803,0.687452,-3.519960,0.000432,0.999998
cort_peak_16179,40.123711,2.754354,0.807089,3.412700,0.000643,0.999998
cort_peak_5971,8.813785,-3.369976,1.003237,-3.359101,0.000782,0.999998
cort_peak_11870,86.298444,2.266453,0.680046,3.332793,0.000860,0.999998
cort_peak_145189,35.197452,2.830571,0.863520,3.277945,0.001046,0.999998
...,...,...,...,...,...,...
cort_peak_179586,3.413964,3.110540,4.663571,0.666987,NaN,NaN
cort_peak_179897,1.573086,2.134140,3.319395,0.642930,NaN,NaN
cort_peak_180546,1.788350,2.313598,3.244107,0.713169,NaN,NaN


In [52]:
## Genes
dds = DeseqDataSet(
    counts=gene_counts,
    clinical=clinical_df,
    design_factors="Annotation",
    refit_cooks=True,
    n_cpus=1,
)
dds

AnnData object with n_obs × n_vars = 6 × 19683
    obs: 'Annotation'
    obsm: 'design_matrix'

In [53]:
dds.deseq2()

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 5.61 seconds.

Fitting dispersion trend curve...
... done in 1.45 seconds.

Fitting MAP dispersions...
... done in 5.60 seconds.

Fitting LFCs...
... done in 2.52 seconds.

Refitting 0 outliers.



In [54]:
stat_res = DeseqStats(dds, n_cpus=2)
stat_res.summary()
gene_results = stat_res.results_df
gene_results = gene_results.sort_values('pvalue', ascending=True)
gene_results.to_csv('../results/ATAC/differential/DAC_gene.csv')
gene_results

Running Wald tests...
... done in 1.79 seconds.

Log2 fold change & Wald test p-value: Annotation Tumor vs Normal


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
TSPAN6,99.692270,0.440064,0.478212,0.920227,0.357454,0.997348
TNMD,33.937291,-0.301201,0.628244,-0.479433,0.631631,0.997348
DPM1,280.865414,0.361631,0.333818,1.083318,0.278667,0.997348
SCYL3,288.181869,0.626359,0.328406,1.907273,0.056485,0.997348
C1orf112,327.849676,-0.392905,0.374722,-1.048524,0.294397,0.997348
...,...,...,...,...,...,...
RP13-210D15.9,10.779070,-0.444944,0.950392,-0.468169,0.639664,0.997348
RP11-244E17.1,362.866479,0.278642,0.332496,0.838033,0.402012,0.997348
CTB-60B18.23,94.591196,-0.107540,0.431677,-0.249122,0.803267,0.997348
RP3-454G6.2,98.269038,0.055708,0.476292,0.116962,0.906890,0.997348


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
LRFN5,332.880470,-1.323594,0.406974,-3.252278,0.001145,0.997348
CRYBG3,259.833503,-1.209481,0.376936,-3.208717,0.001333,0.997348
HMGCS2,46.137177,-1.660528,0.530240,-3.131655,0.001738,0.997348
PRAC2,37.371114,-1.860092,0.598162,-3.109677,0.001873,0.997348
AQP9,75.474269,-1.478217,0.483820,-3.055306,0.002248,0.997348
...,...,...,...,...,...,...
C2CD4C,81.861119,-0.000108,0.442278,-0.000245,0.999805,0.999891
MED23,212.733422,0.000057,0.323466,0.000177,0.999859,0.999891
TOPORS-AS1,274.730846,0.000049,0.359993,0.000137,0.999891,0.999891
C21orf33,4.777863,3.667897,4.652546,0.788363,NaN,NaN


In [55]:
# peak_results[peak_results['pvalue'] < 0.05]#['log2FoldChange'] > 0
gene_results[gene_results['pvalue'] < 0.05]

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
LRFN5,332.880470,-1.323594,0.406974,-3.252278,0.001145,0.997348
CRYBG3,259.833503,-1.209481,0.376936,-3.208717,0.001333,0.997348
HMGCS2,46.137177,-1.660528,0.530240,-3.131655,0.001738,0.997348
PRAC2,37.371114,-1.860092,0.598162,-3.109677,0.001873,0.997348
AQP9,75.474269,-1.478217,0.483820,-3.055306,0.002248,0.997348
...,...,...,...,...,...,...
MID1IP1,151.368361,0.732034,0.373020,1.962451,0.049710,0.997348
CDH19,120.434175,-0.938305,0.478328,-1.961635,0.049805,0.997348
ARRDC5,46.543657,-1.013628,0.516953,-1.960774,0.049905,0.997348
ANXA1,58.475922,-1.177671,0.600673,-1.960585,0.049927,0.997348


In [77]:
## generate peak files for homer analysis
peak_results = pd.read_csv('../results/ATAC/differential/DAC_peaks.csv')
# peak_results.index = peak_results['PeakID']
# peak_results = peak_results.drop(['PeakID'], axis=1)
peak_results

,PeakID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,cort_peak_42326,28.625864,-2.419803,0.687452,-3.519960,0.000432,0.999998
1,cort_peak_16179,40.123711,2.754354,0.807089,3.412700,0.000643,0.999998
2,cort_peak_5971,8.813785,-3.369976,1.003237,-3.359101,0.000782,0.999998
3,cort_peak_11870,86.298444,2.266453,0.680046,3.332793,0.000860,0.999998
4,cort_peak_145189,35.197452,2.830571,0.863520,3.277945,0.001046,0.999998
...,...,...,...,...,...,...,...
175619,cort_peak_179586,3.413964,3.110540,4.663571,0.666987,NaN,NaN
175620,cort_peak_179897,1.573086,2.134140,3.319395,0.642930,NaN,NaN
175621,cort_peak_180546,1.788350,2.313598,3.244107,0.713169,NaN,NaN
175622,cort_peak_180661,2.701899,2.930165,2.796311,1.047868,NaN,NaN


In [78]:
peaks_annotated = pd.read_csv('../results/ATAC/macs/cort_peaks_annotated.narrowPeak', sep='\t')
peaks_annotated = peaks_annotated[['PeakID (cmd=annotatePeaks.pl results/ATAC/macs/cort_peaks.narrowPeak hg38)','Chr','Start','End','Strand','Peak Score','Annotation','Distance to TSS','Nearest Ensembl','Gene Name']]
peaks_annotated.columns = ['PeakID','Chr','Start','End','Strand','Peak Score','Annotation','Distance to TSS','Nearest Ensembl','Gene Name']
# peaks.index = peaks['PeakID']
# peaks = peaks.drop(['PeakID'], axis=1)
peaks_annotated.index = peaks_annotated['PeakID']
peaks_annotated = peaks_annotated.drop(['PeakID'], axis=1)
peaks_annotated

,Chr,Start,End,Strand,Peak Score,Annotation,Distance to TSS,Nearest Ensembl,Gene Name
PeakID,,,,,,,,,
cort_peak_72682,chr17,53105799,53106413,+,1760,Intergenic,120601.0,NaN,C17orf112
cort_peak_116019,chr3,93470333,93470821,+,1648,Intergenic,503504.0,ENSG00000184500,PROS1
cort_peak_49208,chr13,109424086,109424398,+,1567,Intergenic,23546.0,ENSG00000229792,LINC00399
cort_peak_55823,chr15,20505961,20506672,+,1194,promoter-TSS (NR_036432),-136.0,ENSG00000180229,HERC2P3
cort_peak_82351,chr19,20999554,21000284,+,1071,Intergenic,-20738.0,ENSG00000118620,ZNF430
...,...,...,...,...,...,...,...,...,...
cort_peak_167501,chr8,127823141,127823257,+,13,"intron (NR_003367, intron 1 of 8)",27237.0,ENSG00000283710,MIR1204
cort_peak_95046,chr2,178065570,178065783,+,13,"intron (NM_001077197, intron 2 of 20)",7101.0,ENSG00000128655,PDE11A
cort_peak_148764,chr6,139425413,139425558,+,13,Intergenic,49111.0,ENSG00000238099,LINC01625


In [79]:
peak_results_annotated = pd.DataFrame(peak_results)
peak_results_annotated.index = peak_results['PeakID']
peak_results_annotated[['Chr','Start','End','Strand','Peak Score','Annotation','Distance to TSS','Nearest Ensembl','Gene Name']] = peak_results_annotated.apply(lambda x: peaks_annotated.loc[x['PeakID']], axis=1, result_type='expand')
peak_results.index = peak_results['PeakID']
peak_results = peak_results.drop(['PeakID'], axis=1)
peak_results_annotated.to_csv('../results/ATAC/differential/DAC_peaks_annotated.csv')


In [93]:
sig = peak_results_annotated[peak_results_annotated['pvalue']<0.05]
homer_up = sig[sig['log2FoldChange']>0.5][['Chr','Start','End','Strand']]
homer_up.to_csv('../results/ATAC/homer/peaks_up.homer', sep='\t')
homer_down = sig[sig['log2FoldChange']<-0.5][['Chr','Start','End','Strand']]
homer_down.to_csv('../results/ATAC/homer/peaks_down.homer', sep='\t')
homer_up

,Chr,Start,End,Strand
PeakID,,,,
cort_peak_16179,chr1,221839455,221840294,+
cort_peak_11870,chr1,161242446,161243287,+
cort_peak_145189,chr6,75301970,75302682,+
cort_peak_107633,chr22,26132928,26133566,+
cort_peak_71236,chr17,40417608,40418448,+
...,...,...,...,...
cort_peak_177661,chrX,20098824,20099209,+
cort_peak_61310,chr15,92625196,92625906,+
cort_peak_31525,chr11,68030091,68031087,+
